# StructuredOutputParser(구조화된 출력파서)
StructuredOutputParser는 LLM에 대한 답변을 `dict` 형식으로 구성하고, key/value 쌍으로 여러 필드를 반환하고자 할 때 유용하게 사용할 수 있습니다. 

## 장점
Pydantic/JSON 파서가 더 강력하다는 평가를 받지만, StructuredOutputParser는 로컬 모델과 같은 덜 강력한 모델에서도 유용합니다. 이는 GPT나 Claude 모델보다 인텔리전스가 낮은(즉, parameter 수가 적은) 모델에서 특히 효과적입니다. 

## 참고 사항
로컬 모델의 경우 `Pydantic` 파서가 동작하지 않는 상황이 빈번하게 발생할 수 있습니다. 이러한 경우, 대안으로 StructuredOutputParser를 사용하는 것이 좋은 해결책이 될 수 있습니다.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

- `ResponseSchema` 클래스를 사용하여 사용자의 질문에 대한 답변과 사용된 소스(웹사이트)에 대한 설명을 포함하는 응답 스키마를 정의합니다.
- `StructuredOutputParser`를 `response_schemas`를 사용하여 초기화하여, 정의된 응답 스키마에 따라 출력을 구조화합니다.

In [4]:
# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]
#name - 키 description - 키값

# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [6]:
#들어가는 프롬포트
print(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자의 질문에 대한 답변
	"source": string  // 사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.
}
```


In [7]:
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [8]:
model = ChatOpenAI(temperature=0)  # ChatOpenAI 모델 초기화
chain = prompt | model | output_parser  # 프롬프트, 모델, 출력 파서를 연결

In [9]:
# 대한민국의 수도가 무엇인지 질문합니다.
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})

{'answer': '서울', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8'}

### 최종적으로 들어간 입력값
answer the users question as best as possible.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자의 질문에 대한 답변
	"source": string  // 사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.
}
```
대한민국의 수도는 어디인가요?

In [10]:
for s in chain.stream({"question": "세종대왕의 업적은 무엇인가요?"}):
    # 스트리밍 출력
    print(s)

{'answer': '세종대왕은 한글을 창제하고 문화를 발전시키는 등 다양한 업적을 가지고 있습니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95'}


In [11]:
answer = chain.invoke({"question": "대한민국의 수도는 어디인가요?"})
type(answer)

dict

In [12]:
answer["answer"]

'서울'

# JsonOutputParser

JsonOutputParser는 사용자가 원하는 JSON 스키마를 지정할 수 있게 해주는 도구입니다. 이 도구는 Large Language Model (LLM)이 데이터를 조회하고 결과를 도출할 때, 지정된 스키마에 맞게 JSON 형식으로 데이터를 반환할 수 있도록 설계되었습니다.

LLM이 데이터를 정확하고 효율적으로 처리하여 사용자가 원하는 형태의 JSON을 생성하기 위해서는, 모델의 용량(예: 인텔리전스)이 충분히 커야 합니다. 예를 들어, llama-70B 모델은 llama-8B 모델보다 더 큰 용량을 가지고 있어 보다 복잡한 데이터를 처리하는 데 유리합니다.


In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [14]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [15]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [16]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "\uc8fc\uc81c\uc5d0 \ub300\ud55c \uac04\uacb0\ud55c \uc124\uba85", "title": "Description", "type": "string"}, "hashtags": {"description": "\ud574\uc2dc\ud0dc\uadf8 \ud615\uc2dd\uc758 \ud0a4\uc6cc\ub4dc(2\uac1c \uc774\uc0c1)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [17]:
# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인을 구성합니다.
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
answer = chain.invoke({"question": question})

In [18]:
# 타입을 확인합니다.
type(answer)

dict

In [19]:
# answer 객체를 출력합니다.
answer

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 인간 활동에 의해 발생하는 온실가스 배출이 주요 원인입니다. 이는 극지방의 빙하가 녹고 해수면이 상승하며, 기후 패턴이 변화하여 극단적인 기상 현상이 증가하는 등 다양한 환경적 영향을 초래합니다.',
 'hashtags': '#지구온난화 #기후변화'}

**Pydantic 을 사용하지 않고 `JsonOutputParser` 를 사용**

Pydantic 없이도 이 기능을 사용할 수 있습니다. 이 경우 JSON을 반환하도록 요청하지만, 스키마가 어떻게 되어야 하는지에 대한 구체적인 정보는 제공하지 않습니다.

In [20]:
# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)

{'description': '지구 온난화는 대기 중 온실가스 농도의 증가로 인해 지구의 평균 기온이 상승하는 현상입니다. 이는 주로 화석 연료의 연소, 산림 파괴, 산업 활동 등 인간의 활동에 의해 촉발됩니다. 지구 온난화는 기후 변화, 해수면 상승, 극단적인 기상 현상 증가 등 다양한 환경적 문제를 야기합니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#해수면상승']}
